<a href="https://colab.research.google.com/github/arushi019/GA-Based-Neural-Architecture-Prediction/blob/master/CNN_Tut1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms

In [0]:
seed=42
np.random.seed(42)
torch.manual_seed(seed)

In [0]:
#transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
transform=transforms.Compose([transforms.ToTensor()])
train=torchvision.datasets.MNIST(root='',train=True,transform=transform,download=True)
test=torchvision.datasets.MNIST(root='',train=False,transform=transform,download=True)

Processing...
Done!


In [0]:
print(test)
print(train)

Dataset MNIST
    Number of datapoints: 10000
    Split: test
    Root Location: 
    Transforms (if any): Compose(
                             ToTensor()
                         )
    Target Transforms (if any): None
Dataset MNIST
    Number of datapoints: 60000
    Split: train
    Root Location: 
    Transforms (if any): Compose(
                             ToTensor()
                         )
    Target Transforms (if any): None


In [0]:
from torch.utils.data.sampler import SubsetRandomSampler
n_train=50000
train_samp=SubsetRandomSampler(np.arange(n_train))
n_val=10000
val_samp=SubsetRandomSampler(np.arange(n_train,n_train+n_val))
n_test=10000
test_samp=SubsetRandomSampler(np.arange(n_test))


In [0]:
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
class myCNN(nn.Module):
  def __init__(self):
    super(myCNN,self).__init__()
    self.conv1=nn.Conv2d(1,18,kernel_size=3,stride=1,padding=1)
    self.pool=nn.MaxPool2d(kernel_size=2,stride=2,padding=0)
    self.fc1=nn.Linear(18*14*14,64)
    self.fc2=nn.Linear(64,10)
  def forward(self,x):
    x=F.relu(self.conv1(x))
    x=self.pool(x)
    x=x.view(-1,18*14*14)
    x=F.relu(self.fc1(x))
    x=self.fc2(x)
    return x

In [0]:
class customCNN(nn.Sequential):
  #                                               k x s x s         
  #block 1: conv2d+avgPool                       6 x s/2 x s/2            
  #block 2: conv2d+avgPool+dropOut               7 x s/2 x s/2            
  #block 3: reshape+linear+reshape               (1,k*s*s)->64->(1*32*32)
  #block 4: reshape+linear+dropOut+reshape       (1,k*s*s)->64->(1*32*32)
  #reshape+linear layer                          (1,k*s*s)->10
  def block1(k,s):
    l1=nn.Conv2d(k,6,kernel_size=3,stride=1)
    l2=nn.AvgPool2d(kernel_size=2,stride=2)
    return ([l1,l2],6,s/2)
  def block2(k,s):
    l1=nn.Conv2d(k,7,kernel_size=3,stride=1)
    l2=nn.AvgPool2d(kernel_size=2,stride=2)
    l3=nn.Dropout(p=0.5)
    return ([l1,l2,l3],7,s/2)
  def block3(k,s):
    #reshape in forward function
    l1=nn.Linear(k*s*s,64)
    #reshape 
    return ([l1],1,32)
  def block4(k,s):
    #reshape
    l1=nn.Linear(k*s*s,64)
    l2=nn.Dropout(p=0.4)
    #reshape
    return ([l1,l2],1,32)
  def __init__(self):
    super(customCNN,self).__init__()
    self.v=[0,0]
    self.v[0]=np.random.randint(1,5)
    self.v[1]=np.random.randint(1,5)
    self.layers=[]
    k=1
    s=28
    for i in range(2):
      val=0
      if self.v[i]==1:
        val=block1(k,s)
      if self.v[i]==2:
        val=block2(k,s)
      if self.v[i]==3:
        val=block3(k,s)
      if self.v[i]==4:
        val=block4(k,s)
      k=val[1]
      s=val[2]
      for j in range(len(val[0])):
        self.layers.append(val[0][j])
    self.layers=nn.ModuleList(self.layers)